In [2]:
import pandas as pd
from hydra import initialize, compose
import pickle
import numpy as np
from tools.baseline_methods import var_baseline
from tools.baseline_methods import summary_transform, make_human_readable
from causalnex.structure.dynotears import from_numpy_dynamic, from_pandas_dynamic


In [8]:
from gluonts.model.deepar import DeepAREstimator


In [6]:
gluonts.__version__

'0.6.0'

In [5]:
import  gluonts.mx.model.deepar

ModuleNotFoundError: No module named 'gluonts.mx.model.deepar'

In [ ]:
# optimize dynotears? Whats broken?

In [3]:
def dynotears_baseline(data_sample,cfg):
    sm = from_pandas_dynamic(data_sample[:cfg.cut_at].reset_index(drop=True),p=cfg.max_lag)
    print(sm)
    pred = reformat_dynotears(data_sample,sm,cfg.max_lag)
    return summary_transform(pred,cfg)



def reformat_dynotears(data_sample,sm, max_lag):
    pred = list(sm.adjacency())
    # remove lag 0
    pred = [x for x in pred if x[0].split("_")[-1] != "lag0"]
    result = np.zeros((len(data_sample.columns),len(data_sample.columns),max_lag))
    mapping = {x:n for n, x in  enumerate(data_sample.columns)}
    # THIS ASSUMES MISSING LINKS MEAN 0 relationship
    # effect x cause x lag
    for x in pred: 
        # cause
        lag = x[0][-1]
        cause = x[0].split("_")[0]
        # effect: 
        for key in x[1].keys():
            assert key[-1] == "0", "Formatting ISSUE"
            effect = key.split("_")[0]
            value = x[1][key]["weight"]
            result[mapping[effect],mapping[cause], int(lag)-1] = value
    return result


In [ ]:
# Example export of samples

for x in range(10):

    example = ds[x]
    data = load_sample(p= "../" + cfg.data_path, which= example)
    label = graph_to_label_tensor(example, human_readable=True)
    label.to_csv("../datasets/compose_for_wasim/" + str(x) + "_label" + ".csv")
    data.to_csv("../datasets/compose_for_wasim/" + str(x) + "_data"+ ".csv")


In [4]:
# To keep everything clean we use hydra for HPs.
with initialize(version_base=None, config_path="config/"):
    cfg = compose(config_name='additional.yaml')

In [5]:
# To keep everything clean we use hydra for HPs.
with initialize(version_base=None, config_path="config/"):
    cfg2 = compose(config_name='predict_single.yaml')

In [6]:
test_data = pickle.load(open(cfg.sample_path + "synthetic_3" + "/" +  cfg.eval_set + ".p", "rb"))

In [45]:
sample = test_data[0]
data_sample = pd.DataFrame(sample[0].T)
data_sample.columns = ["A", "B", "C"]

In [46]:
sm = from_pandas_dynamic(data_sample[:cfg.method_hps.cut_at].reset_index(drop=True),p=cfg.method_hps.max_lag)


In [48]:
dynotears_baseline(data_sample,cfg.method_hps)

StructureModel with 18 nodes and 31 edges


array([[0.05141952, 0.06535818, 0.13299467],
       [0.39405269, 0.12561292, 0.37163011],
       [0.05871656, 0.11456477, 0.02968097]])

In [47]:

sample[1]

array([[[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]],

       [[0.39322325, 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.31075515, 0.        ]],

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]]])

In [68]:
sample[1].mean(axis=2)

array([[0.        , 0.        , 0.        ],
       [0.13107442, 0.        , 0.10358505],
       [0.        , 0.        , 0.        ]])

In [11]:
sample[1].sum(axis=2)

array([[0.        , 0.        , 0.        ],
       [0.39322325, 0.        , 0.31075515],
       [0.        , 0.        , 0.        ]])

In [17]:
sample[1][1]

array([[0.39322325, 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.31075515, 0.        ]])

In [7]:
from additional_methods.pcmci import pcmci_baseline as baseline_func


[(array([[ 0.        ,  2.49787959,  0.22504948, ..., -1.32073112,
           0.25929744, -0.05349189],
         [ 0.        ,  0.38094664,  1.97057136, ...,  4.11105851,
          -0.83443461,  2.55181412],
         [ 0.        ,  0.50553279,  0.78859098, ...,  1.02978955,
           0.2191926 ,  0.37047718]]),
  array([[[0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ]],
  
         [[0.39322325, 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          [0.        , 0.31075515, 0.        ]],
  
         [[0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ]]])),
 (array([[ 0.        ,  2.70244286,  1.25361348, ...,  0.29467337,
           0.01191895, -0.62962798],
         [ 0.        ,  1.86710031,  1.80869692, ..., -1.14506442,
           0.74209086, -0.05780993],
         [ 0.        , -0.96811918